<a href="https://www.kaggle.com/code/joshuaokolo/object-detection-in-panoramic-images?scriptVersionId=104046127" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import sys
import numpy as np
import cv2
from math import pi, atan, cos, sin, acos, sqrt, tan
from scipy.interpolate import RectBivariateSpline

In [ ]:
def projection_angle(x, d): 
     x_max = (1 + d) / d 
     numerator = -2 * d * x ** 2 + 2 * (d + 1) * sqrt((1 - d ** 2) * x ** 2 + (d + 1) ** 2)
     denominator = 2 * (x ** 2 + (d + 1) ** 2)

     if 0 < x < x_max:
        project_angle = acos(numerator / denominator)
    elif x < 0:
        project_angle = - acos(numerator / denominator)
    elif x == x_max:
        project_angle = pi/2.
    else:
        raise Exception('invalid input args')

    return project_angle

In [ ]:
def panotostereo(panorama, distance):

    frames = []
    input_img = panorama
    height, width, _ = input_img.shape
    d = distance
    xp_max = (1 + d) / d  
    yp_max = (1 + d) / d  
    xp_domain = xp_max * (np.arange(-1., 1., 2. / height) + 1.0 / height)
    yp_domain = yp_max * (np.arange(-1., 1., 2. / height) + 1.0 / height)
    """Get the radian values of each pixel in the image with delta_rad"""
    delta_rad = 2 * pi / width  
    
    for face in range(4):
        print('generating stereo image', face)
        output_img = np.zeros((height, height, 3))
        
        """Use scipy's interpolation function"""
        interpolate_0 = RectBivariateSpline(np.arange(height), np.arange(width), input_img[:, :, 0])
        interpolate_1 = RectBivariateSpline(np.arange(height), np.arange(width), input_img[:, :, 1])
        interpolate_2 = RectBivariateSpline(np.arange(height), np.arange(width), input_img[:, :, 2])
        pano_x = np.zeros((height, 1))
        pano_y = np.zeros((height, 1))
        for j, xp in enumerate(xp_domain):
            phi = projection_angle(xp, d)
            pano_x[j] = (width / 2.0 + (phi / delta_rad))

        for i, yp in enumerate(yp_domain):
            theta = projection_angle(yp, d)
            pano_y[i] = height/2.0 + (theta/delta_rad)

        output_img[:, :, 0] = interpolate_0(pano_y, pano_x)
        output_img[:, :, 1] = interpolate_1(pano_y, pano_x)
        output_img[:, :, 2] = interpolate_2(pano_y, pano_x)

        cv2.imwrite('split_'+str(face)+'_'+str(d)+'.jpg', output_img)
        frames.append(output_img)
        input_img = np.concatenate(
            (input_img[:, int(width/4):, :], input_img[:, :int(width/4), :]), axis=1)
    return frames

In [ ]:
im = cv2.imread('sample.png')  
projection = panotostereo(im, 2)

In [ ]:
!pip3 install tensorflow==2.4.0 keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 
!pip3 install opencv-python keras-resnet==0.2.0 tensorflow-gpu==2.4.0
!pip3 install imageai --upgrade
import glob
import os
import itertools
import cv2
import wget
from imageai.Detection import ObjectDetection
import numpy as np
import pandas as pd

In [ ]:
filenames_input_image = glob.glob(os.path.join('split_*.jpg')) 

"""Check the imported filenames"""
filenames_input_image

In [ ]:
"""By including an if statement, we make sure to not download the detector files if they already exist 
in the home directory of the Python notebook"""

if os.path.isfile("yolo.h5")!=True:
    #download trained yolov3 model if not already downloaded      
    wget.download('https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5') 
if os.path.isfile("resnet50_coco_best_v2.1.0.h5")!=True:        
    wget.download('https://github.com/OlafenwaMoses/ImageAI/releases/download/essentials-v5/resnet50_coco_best_v2.1.0.h5/')

In [ ]:
def imagedetectionyolo():

    detector = ObjectDetection()
    detector.setModelTypeAsYOLOv3() #initialize the model
    detector.setModelPath("yolo.h5")
    detector.loadModel()
    
    detected_object_list = []
    list_counts = []
    question = input("Do you want an image-by-image list of all objects? (y/n)")
    
    
    for image in itertools.islice(filenames_input_image, 0, 10): 
        #loop over input images, limited to 10 for simplicity. the numbers can be changed.
        detection_dict = detector.detectObjectsFromImage(input_image=image, 
                                                         output_image_path=os.path.join(os.path.basename(image)), 
                                                         minimum_percentage_probability=30)  
        #parameter 'minimum_percentage_probability' in the function 'detectObjectsFromImage' is used 
        #to determine the integrity of the detection results. Lowering the value shows more objects while 
        #increasing the value ensures objects with the highest accuracy are detected. 
        
        """The parameter 'minimum_percentage_probability' in the function 'detectObjectsFromImage' above is used 
        to determine the integrity of the detection results (where a high probability of detection for a specific 
        object corresponds to high integrity). Effectively, lowering the minimum percentage value shows more 
        detections of a specific object while increasing the value ensures objects are detected with a higher 
        accuracy. It acts as a threshold to filter out objects that might be ambiguous. For example, the 
        detector may be 10% sure that a Yacht looks like a Car, in which case we can use the threshold to 
        filter it out and save computing time."""

        
        """It is useful in many cases to count the numbers of detected objects for further analysis. 
        We can now use Python’s Pandas library to create lists of counts per unique object for every image, 
        total counts per unique object for all images combined and finally a list of all detected objects. 
        To save computation time, the 'question' variable prompts the user to indicate with yes(y) or no(n) 
        if a list of all objects should be printed."""
        
        detections = pd.DataFrame(detection_dict)
        
        if detections.empty == False:
            detections.index = [os.path.basename(image)] * len(detections)

            df1 = detections['name'].value_counts().rename_axis('object').reset_index(name='counts')
            df1.index = [os.path.basename(image)] * len(df1)
            list_counts.append(df1)

        
            detected_object_list.append(detections)
    list_counts = pd.concat(list_counts) 
    print(" ")
    print(" ")
    print("Counts per unique object for every image")
    print(" ")
    display(list_counts)
    
    detected_object_list = pd.concat(detected_object_list)
    print(" ")
    print(" ")
    print("Total counts per unique object for all images combined")      
    print(" ")
    print(detected_object_list['name'].value_counts())
    
    
    if question == 'y':
        print(" ")
        print(" ")
        print("All detected objects")
        display(detected_object_list)

In [ ]:
imagedetectionyolo()